<a href="https://colab.research.google.com/github/ParishaRaja/Deep_Learning_Projects/blob/main/Car_Price_Prediction(Parisha_Raja_sem7_DS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: Parisha Raja
# Data Science (Sem 7)

In [1]:
# Importing libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Loading data
data = pd.read_csv('car-data.csv')


In [3]:
data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [4]:
data.shape

(6019, 14)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         6019 non-null   int64  
 1   Name               6019 non-null   object 
 2   Location           6019 non-null   object 
 3   Year               6019 non-null   int64  
 4   Kilometers_Driven  6019 non-null   int64  
 5   Fuel_Type          6019 non-null   object 
 6   Transmission       6019 non-null   object 
 7   Owner_Type         6019 non-null   object 
 8   Mileage            6017 non-null   object 
 9   Engine             5983 non-null   object 
 10  Power              5983 non-null   object 
 11  Seats              5977 non-null   float64
 12  New_Price          824 non-null    object 
 13  Price              6019 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 658.5+ KB


In [6]:
data.describe()

,Unnamed: 0,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,3009.000000,2013.358199,5.873838e+04,5.278735,9.479468
std,1737.679967,3.269742,9.126884e+04,0.808840,11.187917
min,0.000000,1998.000000,1.710000e+02,0.000000,0.440000
25%,1504.500000,2011.000000,3.400000e+04,5.000000,3.500000
50%,3009.000000,2014.000000,5.300000e+04,5.000000,5.640000
75%,4513.500000,2016.000000,7.300000e+04,5.000000,9.950000
max,6018.000000,2019.000000,6.500000e+06,10.000000,160.000000


In [7]:
data.duplicated().any()

False

In [8]:
# Handling NaN values with mean imputation for numerical data
data_numeric = data.select_dtypes(include=[np.number])
data_numeric = data_numeric.fillna(data_numeric.mean())

# Handling NaN values with mode imputation for categorical data
data_categorical = data.select_dtypes(include=[np.object])
data_categorical = data_categorical.fillna(data_categorical.mode().iloc[0])

# Combining the numerical and categorical data
data = pd.concat([data_numeric, data_categorical], axis=1)

<ipython-input-8-f553725a1635>:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data_categorical = data.select_dtypes(include=[np.object])


In [9]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
data['Name']= label_encoder.fit_transform(data['Name'])
data['Location']= label_encoder.fit_transform(data['Location'])
data['Fuel_Type']= label_encoder.fit_transform(data['Fuel_Type'])
data['Transmission']= label_encoder.fit_transform(data['Transmission'])
data['Owner_Type']= label_encoder.fit_transform(data['Owner_Type'])
data['Power']= label_encoder.fit_transform(data['Power'])
data['Engine']= label_encoder.fit_transform(data['Engine'])
data['Mileage']= label_encoder.fit_transform(data['Mileage'])
data['New_Price']= label_encoder.fit_transform(data['New_Price'])

In [10]:
x=data.drop(columns=['Price','Unnamed: 0'],axis=1)

In [11]:
y=data['Price']

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [13]:
x_train.shape

(4815, 12)

In [14]:
x_test.shape

(1204, 12)

In [15]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [16]:
model = Sequential()
model.add(Dense(20, activation='relu', input_shape=(12,)))
model.add(Dense(14, activation='relu'))
model.add(Dense(1, activation='relu'))

In [17]:
model.compile(optimizer='adam',loss = 'mean_absolute_error',metrics = ['MAE'])

In [18]:
early_stopping = EarlyStopping(monitor='MAE', patience=3)

In [19]:
model.fit(x_train, y_train,epochs=80, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/80
121/121 [==============================] - 1s 4ms/step - loss: 7.0149 - MAE: 7.0149 - val_loss: 5.7621 - val_MAE: 5.7621
Epoch 2/80
121/121 [==============================] - 0s 3ms/step - loss: 4.3823 - MAE: 4.3823 - val_loss: 4.2446 - val_MAE: 4.2446
Epoch 3/80
121/121 [==============================] - 0s 3ms/step - loss: 3.6076 - MAE: 3.6076 - val_loss: 3.8714 - val_MAE: 3.8714
Epoch 4/80
121/121 [==============================] - 1s 5ms/step - loss: 3.3334 - MAE: 3.3334 - val_loss: 3.6616 - val_MAE: 3.6616
Epoch 5/80
121/121 [==============================] - 1s 5ms/step - loss: 3.1456 - MAE: 3.1456 - val_loss: 3.5253 - val_MAE: 3.5253
Epoch 6/80
121/121 [==============================] - 1s 5ms/step - loss: 3.0214 - MAE: 3.0214 - val_loss: 3.4257 - val_MAE: 3.4257
Epoch 7/80
121/121 [==============================] - 1s 5ms/step - loss: 2.9361 - MAE: 2.9361 - val_loss: 3.3409 - val_MAE: 3.3409
Epoch 8/80
121/121 [==============================] - 1s 4ms/step - loss: 2.

In [20]:
test_loss, test_mae = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

38/38 [==============================] - 0s 1ms/step - loss: 2.0599 - MAE: 2.0599
Test Loss: 2.0598695278167725, Test MAE: 2.0598695278167725


In [21]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

38/38 [==============================] - 0s 2ms/step
